#Download data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ejlok1/cremad")

print("Path to dataset files:", path)

100%|██████████| 451M/451M [00:04<00:00, 108MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ejlok1/cremad/versions/1


In [ ]:
# Download latest version
path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ravdess-emotional-speech-audio


In [ ]:
# Download latest version
path = kagglehub.dataset_download("ejlok1/surrey-audiovisual-expressed-emotion-savee")

print("Path to dataset files:", path)

100%|██████████| 107M/107M [00:00<00:00, 135MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ejlok1/surrey-audiovisual-expressed-emotion-savee/versions/1


In [ ]:
# Download latest version
path = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/toronto-emotional-speech-set-tess


In [ ]:
!cp -r "/root/.cache/kagglehub/datasets/ejlok1/cremad/versions/1" "/content/drive/MyDrive/Colab Notebooks/cremad"
!cp -r "/root/.cache/kagglehub/datasets/ejlok1/surrey-audiovisual-expressed-emotion-savee/versions/1" "/content/drive/MyDrive/Colab Notebooks/savee"
!cp -r "/kaggle/input/ravdess-emotional-speech-audio" "/content/drive/MyDrive/Colab Notebooks/ravdess"
!cp -r "/kaggle/input/toronto-emotional-speech-set-tess" "/content/drive/MyDrive/Colab Notebooks/tess"


#Load data

In [2]:
!pip install audiomentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.11.0
    Uninstalling librosa-0.11.0:
      Successfully uninstalled librosa-0.11.0


In [3]:
import pandas as pd
import numpy as np

import os
import sys

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
Ravdess = "/content/drive/MyDrive/Colab Notebooks/ravdess/audio_speech_actors_01-24"
Crema   = "/content/drive/MyDrive/Colab Notebooks/cremad/AudioWAV"
Tess    = "/content/drive/MyDrive/Colab Notebooks/tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data"
Savee   = "/content/drive/MyDrive/Colab Notebooks/savee/ALL"

In [6]:
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []

for dir in ravdess_directory_list:
    actor_folder = os.path.join(Ravdess, dir)
    if not os.path.isdir(actor_folder):
        continue
    for file in os.listdir(actor_folder):
        if not file.endswith(".wav"):
            continue
        part = file.split('.')[0].split('-')
        file_emotion.append(int(part[2]))
        file_path.append(os.path.join(actor_folder, file))

Ravdess_df = pd.DataFrame({
    "Emotions": file_emotion,
    "Path": file_path
})

Ravdess_df.Emotions.replace({
    1: 'neutral',
    2: 'calm',
    3: 'happy',
    4: 'sad',
    5: 'angry',
    6: 'fear',
    7: 'disgust',
    8: 'surprise'
}, inplace=True)


/tmp/ipython-input-6-2243743617.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Ravdess_df.Emotions.replace({


In [7]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    if not file.endswith(".wav"):
        continue
    full_path = os.path.join(Crema, file)
    file_path.append(full_path)
    part = file.split('_')
    emotion_code = part[2]
    file_emotion.append({
        'SAD': 'sad',
        'ANG': 'angry',
        'DIS': 'disgust',
        'FEA': 'fear',
        'HAP': 'happy',
        'NEU': 'neutral'
    }.get(emotion_code, 'Unknown'))

Crema_df = pd.DataFrame({
    "Emotions": file_emotion,
    "Path": file_path
})


In [8]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    subdir_path = os.path.join(Tess, dir)
    if not os.path.isdir(subdir_path):
        continue
    for file in os.listdir(subdir_path):
        if not file.endswith(".wav"):
            continue
        part = file.split('.')[0].split('_')[2]
        emotion = 'surprise' if part == 'ps' else part
        file_emotion.append(emotion.lower())
        file_path.append(os.path.join(subdir_path, file))

Tess_df = pd.DataFrame({
    "Emotions": file_emotion,
    "Path": file_path
})


In [9]:
savee_directory_list = os.listdir(Savee)

file_emotion = []
file_path = []

for file in savee_directory_list:
    if not file.endswith(".wav"):
        continue
    full_path = os.path.join(Savee, file)
    file_path.append(full_path)
    part = file.split('_')[1]
    ele = part[:-6]
    emotion = {
        'a': 'angry',
        'd': 'disgust',
        'f': 'fear',
        'h': 'happy',
        'n': 'neutral',
        'sa': 'sad'
    }.get(ele, 'surprise')
    file_emotion.append(emotion)

Savee_df = pd.DataFrame({
    "Emotions": file_emotion,
    "Path": file_path
})


In [10]:
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis=0).reset_index(drop=True)

# Check for existence
valid_count = sum(os.path.exists(p) for p in data_path['Path'])
print(f"TOTAL valid paths: {valid_count} / {len(data_path)}")

# Save
data_path.to_csv("/content/drive/MyDrive/Model_Results/data_path.csv", index=False)


TOTAL valid paths: 12162 / 12162


In [11]:
data_path["Emotions"].value_counts()

,count
Emotions,
happy,1923
sad,1923
fear,1923
angry,1923
disgust,1923
neutral,1703
surprise,652
calm,192


In [11]:
def extract_features(data, sample_rate, frame_length=2048, hop_length=512, offset=0.6, duration=2.5):
    """
    Extract hand-crafted features: ZCR, RMSE, Chroma STFT, and MFCC
    """
    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(data, frame_length=frame_length, hop_length=hop_length)
    zcr = np.squeeze(zcr)

    # Root Mean Square Energy
    rmse = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
    rmse = np.squeeze(rmse)

    # MFCC features (13 coefficients as commonly used)
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=13, hop_length=hop_length)
    mfcc = mfcc.flatten()

    # Chroma STFT features (12 pitch classes)
    chroma = librosa.feature.chroma_stft(y=data, sr=sample_rate, hop_length=hop_length)
    chroma = chroma.flatten()

    # Concatenate all features horizontally
    features = np.concatenate([zcr, rmse, mfcc, chroma])

    return features


def noise_augmentation(data, noise_factor=0.035):
    """
    Add Gaussian noise to audio data
    """
    noise = np.random.normal(0, 1, len(data))
    # Scale noise and add to data
    augmented_data = data + noise_factor * noise
    return augmented_data


def pitch_shift_augmentation(data, sample_rate, pitch_factor=0.7):
    """
    Apply pitch shifting to audio data
    """
    # Using librosa's pitch_shift with the specified factor
    # Converting pitch_factor to semitones (0.7 factor ≈ -5.9 semitones)
    semitones = 12 * np.log2(pitch_factor)
    augmented_data = librosa.effects.pitch_shift(data, sr=sample_rate, n_steps=semitones)
    return augmented_data


def load_and_extract_features(file_path, sample_rate=22050, offset=0.6, duration=2.5):
    """
    Load audio file and extract features:
    1. Original audio (O.A.)
    2. Noise-augmented audio (N.A.)
    3. Pitch-shifted audio (P.A.)
    4. Combined noise and pitch-shifted audio (N.A. + P.A.)
    """
    try:
        # Load original audio
        data, _ = librosa.load(file_path, sr=sample_rate, offset=offset, duration=duration)

        # PADDING FIX: Ensure consistent length
        expected_length = int(sample_rate * duration)  # 22050 * 2.5 = 55125 samples

        if len(data) < expected_length:
            # Pad with zeros if audio is shorter than expected
            data = np.pad(data, (0, expected_length - len(data)), mode='constant', constant_values=0)
        elif len(data) > expected_length:
            # Truncate if audio is longer than expected (shouldn't happen with duration param, but safety)
            data = data[:expected_length]

        # Create augmented versions
        noise_data = noise_augmentation(data, noise_factor=0.035)
        pitch_data = pitch_shift_augmentation(data, sample_rate, pitch_factor=0.7)
        combined_data = pitch_shift_augmentation(noise_data, sample_rate, pitch_factor=0.7)

        # Extract features from all versions
        original_features = extract_features(data, sample_rate)
        noise_features = extract_features(noise_data, sample_rate)
        pitch_features = extract_features(pitch_data, sample_rate)
        combined_features = extract_features(combined_data, sample_rate)

        # Concatenate all features (stacking as mentioned in paper)
        final_features = np.concatenate([
            original_features,    # O.A.
            noise_features,       # N.A.
            pitch_features,       # P.A.
            combined_features     # N.A. + P.A.
        ])

        return final_features

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [12]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("GPU Count:", len(tf.config.list_physical_devices('GPU')))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU Count: 1


In [14]:
from joblib import Parallel, delayed
from tqdm import tqdm

def extract_features_safe(file_path, emotion):
    """Safe wrapper for feature extraction"""
    try:
        features = load_and_extract_features(file_path)
        return features, emotion if features is not None else (None, None)
    except:
        return None, None

# Prepare data
data_subset = data_path
file_paths = data_subset['Path'].tolist()
emotions = data_subset['Emotions'].tolist()

# Parallel processing
n_jobs = -1  # Use all available cores
print("Extracting features in parallel...")

results = Parallel(n_jobs=n_jobs)(
    delayed(extract_features_safe)(path, emotion)
    for path, emotion in tqdm(zip(file_paths, emotions), total=len(file_paths))
)

# Collect valid results
X = []
y = []
for features, emotion in results:
    if features is not None:
        X.append(features)
        y.append(emotion)

print(f"Successfully processed {len(X)} files")

100%|██████████| 12162/12162 [54:38<00:00,  3.71it/s]


Successfully processed 12162 files


In [15]:
X1 = np.array(X)
y1 = np.array(y)

In [16]:
save_dir = '/content/drive/MyDrive/Model_Results'
os.makedirs(save_dir, exist_ok=True)

# Save the arrays
np.save(f'{save_dir}/X_features.npy', X1)
np.save(f'{save_dir}/y_labels.npy', y1)

In [13]:
# To load later:
X = np.load('/content/drive/MyDrive/Model_Results/X_features.npy')
y = np.load('/content/drive/MyDrive/Model_Results/y_labels.npy')

In [14]:
X = np.nan_to_num(X, nan=0.0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
#Label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)
print(f"Emotion classes: {le.classes_}")
print(f"Number of classes: {len(le.classes_)}")


Emotion classes: ['angry' 'calm' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
Number of classes: 8


In [16]:
# Split the data (80% train, 10% validation, 10% test)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_scaled, y_categorical, test_size=0.1, random_state=42, stratify=y_categorical
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.111, random_state=42, stratify=y_temp
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 9730 samples
Validation set: 1215 samples
Test set: 1217 samples


In [17]:
# Prepare input shape for 1D CNN
input_shape = (X_train.shape[1], 1)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_reshaped = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

num_classes = y_categorical.shape[1]

print(f"Input shape: {input_shape}")

Input shape: (11664, 1)


In [18]:
from keras.layers import LSTM, Bidirectional

# Build 1D CNN Model (Table 2 from paper)
def build_1d_cnn_model(input_shape, num_classes):
    """
    1D CNN model exactly as described in paper's Table 2
    """
    model = Sequential()

    # Conv Block 1 (no dropout)
    model.add(Conv1D(128, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Conv Block 2 (with dropout)
    model.add(Conv1D(128, kernel_size=5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    # Conv Block 3 (no dropout)
    model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Conv Block 4 (with dropout)
    model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    # Conv Block 5 (with dropout)
    model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    # Dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build CNN_Bi-LSTM Model (Table 2 from paper)
def build_cnn_bilstm_model(input_shape, num_classes):
    """
    CNN_Bi-LSTM model exactly as described in paper's Table 2
    The only difference from CNN model is the addition of Bi-LSTM layer
    """
    model = Sequential()

    # Conv Block 1 (no dropout)
    model.add(Conv1D(128, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Conv Block 2 (with dropout)
    model.add(Conv1D(128, kernel_size=5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    # Conv Block 3 (no dropout)
    model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Conv Block 4 (with dropout)
    model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    # *** Bi-directional LSTM layer ***
    model.add(Bidirectional(LSTM(64, return_sequences=True)))

    # Conv Block 5 (with dropout)
    model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    # Dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    return model


In [19]:
# Build both models
cnn_model = build_1d_cnn_model(input_shape, num_classes)
bilstm_model = build_cnn_bilstm_model(input_shape, num_classes)

# Model summaries
print("\n" + "="*50)
print("1D CNN MODEL SUMMARY")
print("="*50)
cnn_model.summary()

print("\n" + "="*50)
print("CNN_Bi-LSTM MODEL SUMMARY")
print("="*50)
bilstm_model.summary()

# Compile models with paper's exact parameters
optimizer = keras.optimizers.Adam(learning_rate=0.001)

cnn_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

bilstm_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



1D CNN MODEL SUMMARY


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 11664, 128)     │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 11664, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 5832, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 5832, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5832, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 2916, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2916, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 2916, 64)       │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 2916, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 1458, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 1458, 64)       │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1458, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 729, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 729, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 729, 32)        │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 729, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 364, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 364, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 11648)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,491,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,03

 Total params: 1,620,264 (6.18 MB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 11664, 128)     │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 11664, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 5832, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 5832, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 5832, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 2916, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 2916, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 2916, 64)       │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 2916, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 1458, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 1458, 64)       │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 1458, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 729, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 729, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 729, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 729, 32)        │        12,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 729, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 364, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 364, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 11648)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,491,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 1,692,456 (6.46 MB)

 Trainable params: 1,691,368 (6.45 MB)

 Non-trainable params: 1,088 (4.25 KB)

In [20]:
!pip install mlflow dagshub

import mlflow
import dagshub

# Initialize DagsHub (replace with your actual repo details)
dagshub.init(repo_owner="dimna21", repo_name="Emotion_Recognition", mlflow=True)

# Set MLflow tracking URI to DagsHub
mlflow.set_tracking_uri("https://dagshub.com/dimna21/Emotion_Recognition.mlflow")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=09774212-bad1-41d9-bffb-ebe1905c97ea&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=777b3ab261d0123bd1c46fc78103aabeab8913ffc3b4ab08d1d088c1d2840443




Accessing as dimna21

Initialized MLflow to track repo "dimna21/Emotion_Recognition"

Repository dimna21/Emotion_Recognition initialized!

In [ ]:
import mlflow.keras
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
import seaborn as sns
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import os

tf.config.run_functions_eagerly(True)

# Training parameters
batch_size = 64
epochs = 100

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    factor=0.5,
    min_lr=0.00001,
    verbose=1
)

callbacks = [early_stopping, lr_scheduler]

# Start MLflow experiment for CNN
with mlflow.start_run(run_name="cnn_speech_emotion_recognition"):

    # Log parameters
    mlflow.log_params({
        "model_type": "CNN",
        "batch_size": batch_size,
        "epochs": epochs,
        "learning_rate": 0.001,
        "num_samples": len(X_train),
        "num_features": X_train.shape[1],
        "num_classes": num_classes,
        "early_stopping_patience": 5,
        "lr_scheduler_patience": 3
    })

    # Create and compile CNN model
    cnn_optimizer = keras.optimizers.Adam(learning_rate=0.001)
    cnn_model.compile(
        optimizer=cnn_optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train CNN model
    print("Training CNN model...")
    cnn_history = cnn_model.fit(
        X_train_reshaped, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val_reshaped, y_val),
        callbacks=callbacks,
        verbose=1
    )

    # Log CNN metrics
    mlflow.log_metrics({
        "final_train_acc": cnn_history.history['accuracy'][-1],
        "final_val_acc": cnn_history.history['val_accuracy'][-1],
        "final_train_loss": cnn_history.history['loss'][-1],
        "final_val_loss": cnn_history.history['val_loss'][-1]
    })

    # Evaluate CNN on test set
    cnn_pred_proba = cnn_model.predict(X_test_reshaped)
    y_true = np.argmax(y_test, axis=1)
    cnn_pred = np.argmax(cnn_pred_proba, axis=1)

    cnn_test_acc = accuracy_score(y_true, cnn_pred)
    cnn_f1_weighted = f1_score(y_true, cnn_pred, average='weighted')
    cnn_f1_per_class = f1_score(y_true, cnn_pred, average=None)

    # Log CNN test results
    mlflow.log_metrics({
        "test_accuracy": cnn_test_acc,
        "test_f1_weighted": cnn_f1_weighted
    })

    # Log per-emotion F1 scores
    emotion_classes = le.classes_
    cnn_f1_dict = {f"f1_{emotion}": score for emotion, score in zip(emotion_classes, cnn_f1_per_class)}
    mlflow.log_metrics(cnn_f1_dict)

    # Create CNN confusion matrix
    plt.figure(figsize=(10, 8))
    cm_cnn = confusion_matrix(y_true, cnn_pred)
    sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('CNN Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig('cnn_confusion_matrix.png', dpi=300, bbox_inches='tight')
    mlflow.log_artifact('cnn_confusion_matrix.png')
    plt.show()

    # Plot CNN training history
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].plot(cnn_history.history['accuracy'], label='Train')
    axes[0].plot(cnn_history.history['val_accuracy'], label='Val')
    axes[0].set_title('CNN Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(cnn_history.history['loss'], label='Train')
    axes[1].plot(cnn_history.history['val_loss'], label='Val')
    axes[1].set_title('CNN Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig('cnn_training_history.png', dpi=300, bbox_inches='tight')
    mlflow.log_artifact('cnn_training_history.png')
    plt.show()

    # Save CNN model
    models_dir = '/content/drive/MyDrive/Model_Results'
    os.makedirs(models_dir, exist_ok=True)
    cnn_model.save(f'{models_dir}/cnn_model.keras')
    mlflow.log_artifact(f'{models_dir}/cnn_model.keras')

    print(f"CNN Training Complete!")
    print(f"CNN Test Accuracy: {cnn_test_acc:.4f}, F1: {cnn_f1_weighted:.4f}")

In [ ]:
# Start MLflow experiment for BiLSTM
with mlflow.start_run(run_name="bilstm_speech_emotion_recognition"):

    # Log parameters
    mlflow.log_params({
        "model_type": "CNN_BiLSTM",
        "batch_size": batch_size,
        "epochs": epochs,
        "learning_rate": 0.001,
        "num_samples": len(X_train),
        "num_features": X_train.shape[1],
        "num_classes": num_classes,
        "early_stopping_patience": 5,
        "lr_scheduler_patience": 3
    })

    # Create and compile BiLSTM model
    bilstm_optimizer = keras.optimizers.Adam(learning_rate=0.001)
    bilstm_model.compile(
        optimizer=bilstm_optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train BiLSTM model
    print("Training CNN_BiLSTM model...")
    bilstm_history = bilstm_model.fit(
        X_train_reshaped, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val_reshaped, y_val),
        callbacks=callbacks,
        verbose=1
    )

    # Log BiLSTM metrics
    mlflow.log_metrics({
        "final_train_acc": bilstm_history.history['accuracy'][-1],
        "final_val_acc": bilstm_history.history['val_accuracy'][-1],
        "final_train_loss": bilstm_history.history['loss'][-1],
        "final_val_loss": bilstm_history.history['val_loss'][-1]
    })

    # Evaluate BiLSTM on test set
    bilstm_pred_proba = bilstm_model.predict(X_test_reshaped)
    bilstm_pred = np.argmax(bilstm_pred_proba, axis=1)

    bilstm_test_acc = accuracy_score(y_true, bilstm_pred)
    bilstm_f1_weighted = f1_score(y_true, bilstm_pred, average='weighted')
    bilstm_f1_per_class = f1_score(y_true, bilstm_pred, average=None)

    # Log BiLSTM test results
    mlflow.log_metrics({
        "test_accuracy": bilstm_test_acc,
        "test_f1_weighted": bilstm_f1_weighted
    })

    # Log per-emotion F1 scores
    bilstm_f1_dict = {f"f1_{emotion}": score for emotion, score in zip(emotion_classes, bilstm_f1_per_class)}
    mlflow.log_metrics(bilstm_f1_dict)

    # Create BiLSTM confusion matrix
    plt.figure(figsize=(10, 8))
    cm_bilstm = confusion_matrix(y_true, bilstm_pred)
    sns.heatmap(cm_bilstm, annot=True, fmt='d', cmap='Greens',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('CNN_BiLSTM Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig('bilstm_confusion_matrix.png', dpi=300, bbox_inches='tight')
    mlflow.log_artifact('bilstm_confusion_matrix.png')
    plt.show()

    # Plot BiLSTM training history
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].plot(bilstm_history.history['accuracy'], label='Train')
    axes[0].plot(bilstm_history.history['val_accuracy'], label='Val')
    axes[0].set_title('CNN_BiLSTM Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(bilstm_history.history['loss'], label='Train')
    axes[1].plot(bilstm_history.history['val_loss'], label='Val')
    axes[1].set_title('CNN_BiLSTM Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig('bilstm_training_history.png', dpi=300, bbox_inches='tight')
    mlflow.log_artifact('bilstm_training_history.png')
    plt.show()

    # Save BiLSTM model
    bilstm_model.save(f'{models_dir}/bilstm_model.keras')
    mlflow.log_artifact(f'{models_dir}/bilstm_model.keras')

    print(f"BiLSTM Training Complete!")
    print(f"BiLSTM Test Accuracy: {bilstm_test_acc:.4f}, F1: {bilstm_f1_weighted:.4f}")

In [ ]:
# Start MLflow experiment for Ensemble
with mlflow.start_run(run_name="ensemble_speech_emotion_recognition"):

    # Log parameters
    mlflow.log_params({
        "model_type": "Ensemble_CNN_BiLSTM",
        "ensemble_method": "averaging",
        "num_samples": len(X_train),
        "num_features": X_train.shape[1],
        "num_classes": num_classes
    })

    # Create ensemble predictions (averaging)
    ensemble_pred_proba = (cnn_pred_proba + bilstm_pred_proba) / 2
    ensemble_pred = np.argmax(ensemble_pred_proba, axis=1)

    # Calculate ensemble metrics
    ensemble_test_acc = accuracy_score(y_true, ensemble_pred)
    ensemble_f1_weighted = f1_score(y_true, ensemble_pred, average='weighted')
    ensemble_f1_per_class = f1_score(y_true, ensemble_pred, average=None)

    # Log ensemble test results
    mlflow.log_metrics({
        "test_accuracy": ensemble_test_acc,
        "test_f1_weighted": ensemble_f1_weighted
    })

    # Log per-emotion F1 scores
    ensemble_f1_dict = {f"f1_{emotion}": score for emotion, score in zip(emotion_classes, ensemble_f1_per_class)}
    mlflow.log_metrics(ensemble_f1_dict)

    # Create comprehensive comparison plots
    plt.figure(figsize=(20, 15))

    # CNN Confusion Matrix
    plt.subplot(2, 3, 1)
    cm_cnn = confusion_matrix(y_true, cnn_pred)
    sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('CNN Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # BiLSTM Confusion Matrix
    plt.subplot(2, 3, 2)
    cm_bilstm = confusion_matrix(y_true, bilstm_pred)
    sns.heatmap(cm_bilstm, annot=True, fmt='d', cmap='Greens',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('CNN_BiLSTM Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # Ensemble Confusion Matrix
    plt.subplot(2, 3, 3)
    cm_ensemble = confusion_matrix(y_true, ensemble_pred)
    sns.heatmap(cm_ensemble, annot=True, fmt='d', cmap='Oranges',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('Ensemble Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # F1 Score Comparison Bar Plot
    plt.subplot(2, 3, 4)
    x_pos = np.arange(len(emotion_classes))
    width = 0.25

    plt.bar(x_pos - width, cnn_f1_per_class, width, label='CNN', alpha=0.8)
    plt.bar(x_pos, bilstm_f1_per_class, width, label='CNN_BiLSTM', alpha=0.8)
    plt.bar(x_pos + width, ensemble_f1_per_class, width, label='Ensemble', alpha=0.8)

    plt.xlabel('Emotions')
    plt.ylabel('F1 Score')
    plt.title('Per-Emotion F1 Scores Comparison')
    plt.xticks(x_pos, emotion_classes, rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Overall Performance Comparison
    plt.subplot(2, 3, 5)
    models = ['CNN', 'CNN_BiLSTM', 'Ensemble']
    accuracies = [cnn_test_acc, bilstm_test_acc, ensemble_test_acc]
    f1_scores = [cnn_f1_weighted, bilstm_f1_weighted, ensemble_f1_weighted]

    x_pos = np.arange(len(models))
    width = 0.35

    plt.bar(x_pos - width/2, accuracies, width, label='Accuracy', alpha=0.8)
    plt.bar(x_pos + width/2, f1_scores, width, label='F1 Score', alpha=0.8)

    plt.xlabel('Models')
    plt.ylabel('Score')
    plt.title('Overall Performance Comparison')
    plt.xticks(x_pos, models)
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('ensemble_comparison.png', dpi=300, bbox_inches='tight')
    mlflow.log_artifact('ensemble_comparison.png')
    plt.show()

    # Create detailed classification reports
    classification_reports = {
        'cnn': classification_report(y_true, cnn_pred, target_names=emotion_classes, output_dict=True),
        'bilstm': classification_report(y_true, bilstm_pred, target_names=emotion_classes, output_dict=True),
        'ensemble': classification_report(y_true, ensemble_pred, target_names=emotion_classes, output_dict=True)
    }

    with open('classification_reports.json', 'w') as f:
        json.dump(classification_reports, f, indent=2)
    mlflow.log_artifact('classification_reports.json')

    # Create summary metrics table
    summary_data = {
        'emotion': list(emotion_classes),
        'cnn_f1': list(cnn_f1_per_class),
        'bilstm_f1': list(bilstm_f1_per_class),
        'ensemble_f1': list(ensemble_f1_per_class)
    }

    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv('per_emotion_f1_scores.csv', index=False)
    mlflow.log_artifact('per_emotion_f1_scores.csv')

    print("Ensemble Evaluation Complete!")
    print(f"\nFINAL RESULTS:")
    print(f"CNN Test Accuracy: {cnn_test_acc:.4f}, F1: {cnn_f1_weighted:.4f}")
    print(f"BiLSTM Test Accuracy: {bilstm_test_acc:.4f}, F1: {bilstm_f1_weighted:.4f}")
    print(f"Ensemble Test Accuracy: {ensemble_test_acc:.4f}, F1: {ensemble_f1_weighted:.4f}")

In [21]:
import mlflow.keras
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
import seaborn as sns
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

tf.config.run_functions_eagerly(True)

# Training parameters
batch_size = 128
epochs = 100

# Callbacks from paper
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    factor=0.5,
    min_lr=0.00001,
    verbose=1
)

callbacks = [early_stopping, lr_scheduler]

# Create separate optimizers for each model
cnn_optimizer = keras.optimizers.Adam(learning_rate=0.001)
bilstm_optimizer = keras.optimizers.Adam(learning_rate=0.001)


# Start MLflow experiment
with mlflow.start_run(run_name="speech_emotion_recognition"):

    # Log parameters
    mlflow.log_params({
        "batch_size": batch_size,
        "epochs": epochs,
        "learning_rate": 0.001,
        "num_samples": len(X_train),
        "num_features": X_train.shape[1],
        "num_classes": num_classes,
        "early_stopping_patience": 5,
        "lr_scheduler_patience": 3
    })

    # Train CNN model
    print("Training CNN model...")
    cnn_history = cnn_model.fit(
        X_train_reshaped, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val_reshaped, y_val),
        callbacks=callbacks,
        verbose=1
    )

    # Log CNN metrics
    mlflow.log_metrics({
        "cnn_final_train_acc": cnn_history.history['accuracy'][-1],
        "cnn_final_val_acc": cnn_history.history['val_accuracy'][-1],
        "cnn_final_train_loss": cnn_history.history['loss'][-1],
        "cnn_final_val_loss": cnn_history.history['val_loss'][-1]
    })

    # Train BiLSTM model
    print("Training BiLSTM model...")
    bilstm_history = bilstm_model.fit(
        X_train_reshaped, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val_reshaped, y_val),
        callbacks=callbacks,
        verbose=1
    )

    # Log BiLSTM metrics
    mlflow.log_metrics({
        "bilstm_final_train_acc": bilstm_history.history['accuracy'][-1],
        "bilstm_final_val_acc": bilstm_history.history['val_accuracy'][-1],
        "bilstm_final_train_loss": bilstm_history.history['loss'][-1],
        "bilstm_final_val_loss": bilstm_history.history['val_loss'][-1]
    })

    # Evaluate and log test results
    cnn_pred_proba = cnn_model.predict(X_test_reshaped)
    bilstm_pred_proba = bilstm_model.predict(X_test_reshaped)
    ensemble_pred_proba = (cnn_pred_proba + bilstm_pred_proba) / 2

    # Convert predictions and true labels
    y_true = np.argmax(y_test, axis=1)
    cnn_pred = np.argmax(cnn_pred_proba, axis=1)
    bilstm_pred = np.argmax(bilstm_pred_proba, axis=1)
    ensemble_pred = np.argmax(ensemble_pred_proba, axis=1)

    # Calculate test accuracies
    cnn_test_acc = accuracy_score(y_true, cnn_pred)
    bilstm_test_acc = accuracy_score(y_true, bilstm_pred)
    ensemble_test_acc = accuracy_score(y_true, ensemble_pred)

    # Calculate weighted F1 scores
    cnn_f1_weighted = f1_score(y_true, cnn_pred, average='weighted')
    bilstm_f1_weighted = f1_score(y_true, bilstm_pred, average='weighted')
    ensemble_f1_weighted = f1_score(y_true, ensemble_pred, average='weighted')

    # Calculate per-class F1 scores
    cnn_f1_per_class = f1_score(y_true, cnn_pred, average=None)
    bilstm_f1_per_class = f1_score(y_true, bilstm_pred, average=None)
    ensemble_f1_per_class = f1_score(y_true, ensemble_pred, average=None)

    # Log test accuracies and weighted F1 scores
    mlflow.log_metrics({
        "cnn_test_accuracy": cnn_test_acc,
        "bilstm_test_accuracy": bilstm_test_acc,
        "ensemble_test_accuracy": ensemble_test_acc,
        "cnn_f1_weighted": cnn_f1_weighted,
        "bilstm_f1_weighted": bilstm_f1_weighted,
        "ensemble_f1_weighted": ensemble_f1_weighted
    })

    # Log per-emotion F1 scores for each model
    emotion_classes = le.classes_

    # CNN per-emotion F1 scores
    cnn_f1_dict = {f"cnn_f1_{emotion}": score for emotion, score in zip(emotion_classes, cnn_f1_per_class)}
    mlflow.log_metrics(cnn_f1_dict)

    # BiLSTM per-emotion F1 scores
    bilstm_f1_dict = {f"bilstm_f1_{emotion}": score for emotion, score in zip(emotion_classes, bilstm_f1_per_class)}
    mlflow.log_metrics(bilstm_f1_dict)

    # Ensemble per-emotion F1 scores
    ensemble_f1_dict = {f"ensemble_f1_{emotion}": score for emotion, score in zip(emotion_classes, ensemble_f1_per_class)}
    mlflow.log_metrics(ensemble_f1_dict)

    # Create and save confusion matrices
    plt.figure(figsize=(20, 15))

    # CNN Confusion Matrix
    plt.subplot(2, 3, 1)
    cm_cnn = confusion_matrix(y_true, cnn_pred)
    sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('CNN Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # BiLSTM Confusion Matrix
    plt.subplot(2, 3, 2)
    cm_bilstm = confusion_matrix(y_true, bilstm_pred)
    sns.heatmap(cm_bilstm, annot=True, fmt='d', cmap='Greens',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('BiLSTM Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # Ensemble Confusion Matrix
    plt.subplot(2, 3, 3)
    cm_ensemble = confusion_matrix(y_true, ensemble_pred)
    sns.heatmap(cm_ensemble, annot=True, fmt='d', cmap='Oranges',
                xticklabels=emotion_classes, yticklabels=emotion_classes)
    plt.title('Ensemble Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # F1 Score Comparison Bar Plot
    plt.subplot(2, 3, 4)
    x_pos = np.arange(len(emotion_classes))
    width = 0.25

    plt.bar(x_pos - width, cnn_f1_per_class, width, label='CNN', alpha=0.8)
    plt.bar(x_pos, bilstm_f1_per_class, width, label='BiLSTM', alpha=0.8)
    plt.bar(x_pos + width, ensemble_f1_per_class, width, label='Ensemble', alpha=0.8)

    plt.xlabel('Emotions')
    plt.ylabel('F1 Score')
    plt.title('Per-Emotion F1 Scores Comparison')
    plt.xticks(x_pos, emotion_classes, rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Overall Performance Comparison
    plt.subplot(2, 3, 5)
    models = ['CNN', 'BiLSTM', 'Ensemble']
    accuracies = [cnn_test_acc, bilstm_test_acc, ensemble_test_acc]
    f1_scores = [cnn_f1_weighted, bilstm_f1_weighted, ensemble_f1_weighted]

    x_pos = np.arange(len(models))
    width = 0.35

    plt.bar(x_pos - width/2, accuracies, width, label='Accuracy', alpha=0.8)
    plt.bar(x_pos + width/2, f1_scores, width, label='F1 Score', alpha=0.8)

    plt.xlabel('Models')
    plt.ylabel('Score')
    plt.title('Overall Performance Comparison')
    plt.xticks(x_pos, models)
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('confusion_matrices_and_metrics.png', dpi=300, bbox_inches='tight')
    mlflow.log_artifact('confusion_matrices_and_metrics.png')

    # Save and log models
    cnn_model.save('cnn_model.h5')
    bilstm_model.save('bilstm_model.h5')

    # Log the saved model files as artifacts
    mlflow.log_artifact('cnn_model.h5')
    mlflow.log_artifact('bilstm_model.h5')

    # Log training plots as artifacts
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # CNN plots
    axes[0,0].plot(cnn_history.history['accuracy'], label='Train')
    axes[0,0].plot(cnn_history.history['val_accuracy'], label='Val')
    axes[0,0].set_title('CNN Accuracy')
    axes[0,0].legend()
    axes[0,0].grid(True)

    axes[0,1].plot(cnn_history.history['loss'], label='Train')
    axes[0,1].plot(cnn_history.history['val_loss'], label='Val')
    axes[0,1].set_title('CNN Loss')
    axes[0,1].legend()
    axes[0,1].grid(True)

    # BiLSTM plots
    axes[1,0].plot(bilstm_history.history['accuracy'], label='Train')
    axes[1,0].plot(bilstm_history.history['val_accuracy'], label='Val')
    axes[1,0].set_title('BiLSTM Accuracy')
    axes[1,0].legend()
    axes[1,0].grid(True)

    axes[1,1].plot(bilstm_history.history['loss'], label='Train')
    axes[1,1].plot(bilstm_history.history['val_loss'], label='Val')
    axes[1,1].set_title('BiLSTM Loss')
    axes[1,1].legend()
    axes[1,1].grid(True)

    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    mlflow.log_artifact('training_history.png')

    # Create detailed classification reports and save as JSON
    classification_reports = {
        'cnn': classification_report(y_true, cnn_pred, target_names=emotion_classes, output_dict=True),
        'bilstm': classification_report(y_true, bilstm_pred, target_names=emotion_classes, output_dict=True),
        'ensemble': classification_report(y_true, ensemble_pred, target_names=emotion_classes, output_dict=True)
    }

    with open('classification_reports.json', 'w') as f:
        json.dump(classification_reports, f, indent=2)
    mlflow.log_artifact('classification_reports.json')

    # Create summary metrics table
    summary_data = {
        'emotion': list(emotion_classes),
        'cnn_f1': list(cnn_f1_per_class),
        'bilstm_f1': list(bilstm_f1_per_class),
        'ensemble_f1': list(ensemble_f1_per_class)
    }

    import pandas as pd
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv('per_emotion_f1_scores.csv', index=False)
    mlflow.log_artifact('per_emotion_f1_scores.csv')

    print("Experiment logged to DagsHub!")
    print(f"CNN Test Accuracy: {cnn_test_acc:.4f}, F1: {cnn_f1_weighted:.4f}")
    print(f"BiLSTM Test Accuracy: {bilstm_test_acc:.4f}, F1: {bilstm_f1_weighted:.4f}")
    print(f"Ensemble Test Accuracy: {ensemble_test_acc:.4f}, F1: {ensemble_f1_weighted:.4f}")

Training CNN model...


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 61s 664ms/step - accuracy: 0.4379 - loss: 1.6693 - val_accuracy: 0.1498 - val_loss: 2.8154 - learning_rate: 0.0010
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 50s 646ms/step - accuracy: 0.6417 - loss: 0.9697 - val_accuracy: 0.1531 - val_loss: 2.7730 - learning_rate: 0.0010
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 81s 634ms/step - accuracy: 0.7145 - loss: 0.7942 - val_accuracy: 0.2041 - val_loss: 3.0740 - learning_rate: 0.0010
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 49s 643ms/step - accuracy: 0.7782 - loss: 0.6247 - val_accuracy: 0.2790 - val_loss: 2.5800 - learning_rate: 0.0010
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 82s 643ms/step - accuracy: 0.8288 - loss: 0.5018 - val_accuracy: 0.3967 - val_loss: 2.0809 - learning_rate: 0.0010
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 82s 643ms/step - accuracy: 0.8415 - loss: 0.4662 - val_accuracy: 0.5679 - val_loss: 1.3441 - learning_rate: 0.0010
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 49s 635ms/step - accuracy: 0.8210 - l

KeyboardInterrupt: 

In [ ]:
import os

models_dir = '/content/drive/MyDrive/Model_Results'
os.makedirs(models_dir, exist_ok=True)

print("Saving models to Google Drive...")
cnn_model.save(f'{models_dir}/cnn_model.h5')
bilstm_model.save(f'{models_dir}/bilstm_model.h5')

Saving models to Google Drive...
